Perguntas: 
1-número de fatalidades tem diminuido com os anos? *procure pelas colunas de fataliade
2-durante a noite há mais ocorrências (18:00 a 05:00)?
3-algumas regiões tem mais ocorrências que outras?
4-Verificar em quantas cidades já aconteceram ocorrências?
5-alguns operadores têm mais ocorrências que outros?

In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, hour
spark = SparkSession.builder.appName("MeuApp").getOrCreate()

In [ ]:
df_ocorrencias = spark.read.option("delimiter", ";").option("header", True).csv("ocorrencia.csv")
df_ocorrencias.show(truncate=False)

In [ ]:
df_municipios = spark.read.option("delimiter", ",").option("header", True).csv("municipios.csv")
df_municipios.show(truncate=False)

In [45]:
df_estados = spark.read.option("delimiter",",").option("header",True).csv("estados.csv")
df_estados.show(truncate=False, n=50
)

+---------+---+-------------------+--------+---------+------------+
|codigo_uf|uf |nome               |latitude|longitude|regiao      |
+---------+---+-------------------+--------+---------+------------+
|11       |RO |Rondônia           |-10.83  |-63.34   |Norte       |
|12       |AC |Acre               |-8.77   |-70.55   |Norte       |
|13       |AM |Amazonas           |-3.47   |-65.1    |Norte       |
|14       |RR |Roraima            |1.99    |-61.33   |Norte       |
|15       |PA |Pará               |-3.79   |-52.48   |Norte       |
|16       |AP |Amapá              |1.41    |-51.77   |Norte       |
|17       |TO |Tocantins          |-9.46   |-48.26   |Norte       |
|21       |MA |Maranhão           |-5.42   |-45.44   |Nordeste    |
|22       |PI |Piauí              |-6.6    |-42.28   |Nordeste    |
|23       |CE |Ceará              |-5.2    |-39.53   |Nordeste    |
|24       |RN |Rio Grande do Norte|-5.81   |-36.59   |Nordeste    |
|25       |PB |Paraíba            |-7.28   |-36.

Pergunta 2:
Durante a noite há mais ocorrências (18:00 a 05:00)?

In [ ]:
# Definir uma função que verifica o período do dia
def periodo_dia(hora_col):
    return F.when(
        hora_col.isNull(), F.lit(None)
    ).when(
        (hora_col >= 18) | (hora_col < 5), F.lit('noite')
    ).otherwise(F.lit('dia'))

In [ ]:
# Adicionar a nova coluna 'Periodo_do_Dia' baseada na coluna 'Hora_da_Ocorrencia'
df_ocorrencias = df_ocorrencias.withColumn('Periodo_do_Dia', periodo_dia(F.hour('Hora_da_Ocorrencia')))

# Mostrar o DataFrame resultante
#df_ocorrencias = df_ocorrencias.select("Numero_da_Ocorrencia","Classificacao_da_Ocorrencia","Hora_da_Ocorrencia","Periodo_do_Dia")
df_ocorrencias.show(truncate=False)

In [ ]:
df_periodoDia = df_ocorrencias.groupBy("Periodo_do_Dia").agg(F.count(F.col("Periodo_do_Dia")).alias("Total"))
df_periodoDia.filter((F.col("Periodo_do_Dia") != "NULL")).show(truncate=False)
#df_periodoDia.show(truncate=False)

Pergunta 3:
Algumas regiões tem mais ocorrências que outras?


In [46]:

# Realizar o join entre df_ocorrencias e df_outras_regioes baseado na coluna "Regiao"
df_joined = df_ocorrencias.join(df_estados, "Regiao", "inner")

# Agora, realizar a agregação no DataFrame resultante do join
df_regioes = df_joined.groupBy("Regiao").agg(F.count(F.col("Regiao")).alias("Qtd_ocorrencias"))

# Exibir o resultado
df_regioes.show()


+------------+---------------+
|      Regiao|Qtd_ocorrencias|
+------------+---------------+
|    Nordeste|          13221|
|         Sul|           7113|
|     Sudeste|          22324|
|Centro-Oeste|           7752|
|       Norte|          10311|
+------------+---------------+

